In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
from PIL import Image
from io import BytesIO
from torchvision import transforms
import torch
from tqdm import tqdm  
import time            
import torchvision.transforms.functional as TF 
from PIL import Image, ImageOps

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
